In [ ]:
!pip install -U ultralytics
!pip install pyswarms
!pip install opencv-python matplotlib seaborn

: 

In [ ]:
import os
import torch
import numpy as np
import matplotlib.pyplot as plt

from ultralytics import YOLO
import pyswarms as ps

print("PyTorch:", torch.__version__)
print("CUDA Available:", torch.cuda.is_available())

In [ ]:
DATASET_PATH = "/home/boss/Personal Projects/Aruna Client/pcb-defect-dataset"
DATA_YAML = os.path.join(DATASET_PATH, "data.yaml")

assert os.path.exists(DATA_YAML), "data.yaml not found!"
print("Dataset path verified ✅")


In [ ]:
def count_images(split):
    img_dir = os.path.join(DATASET_PATH, split, "images")
    return len([f for f in os.listdir(img_dir) if f.endswith((".jpg", ".png"))])

for split in ["train", "val", "test"]:
    print(f"{split.upper()} images:", count_images(split))


In [ ]:
model = YOLO("yolov10n.pt")
model.info()


In [ ]:
baseline_results = model.train(
    data=DATA_YAML,
    epochs=50,
    imgsz=640,
    batch=16,
    optimizer="AdamW",
    lr0=0.001,
    weight_decay=0.0005,
    hsv_h=0.015,
    hsv_s=0.7,
    hsv_v=0.4,
    degrees=10,
    flipud=0.5,
    fliplr=0.5,
    mosaic=1.0,
    patience=20,
    project="pcb_yolov10",
    name="baseline"
)


In [ ]:
baseline_metrics = model.val(data=DATA_YAML)
print(baseline_metrics)


In [ ]:
def pso_fitness_function(particles):
    scores = []

    for i, p in enumerate(particles):
        lr, momentum, wd = p

        print(f"Particle {i+1}: lr={lr:.6f}, momentum={momentum:.3f}, wd={wd:.6f}")

        model = YOLO("yolov10n.pt")

        results = model.train(
            data=DATA_YAML,
            epochs=20,              # short runs for PSO
            imgsz=640,
            batch=16,
            optimizer="SGD",
            lr0=float(lr),
            momentum=float(momentum),
            weight_decay=float(wd),
            verbose=False
        )

        mAP50 = results.results_dict.get("metrics/mAP50", 0.0)
        scores.append(-mAP50)       # PSO minimizes

    return scores


In [ ]:
bounds = (
    np.array([1e-4, 0.70, 1e-5]),   # min: lr, momentum, wd
    np.array([1e-2, 0.95, 1e-3])    # max
)

options = {
    'c1': 1.5,   # cognitive
    'c2': 1.5,   # social
    'w': 0.7     # inertia
}

optimizer = ps.single.GlobalBestPSO(
    n_particles=5,
    dimensions=3,
    options=options,
    bounds=bounds
)


In [ ]:
best_cost, best_pos = optimizer.optimize(pso_fitness_function, iters=5)

best_lr, best_momentum, best_wd = best_pos

print("\n✅ PSO OPTIMIZED PARAMETERS")
print("Learning Rate :", best_lr)
print("Momentum      :", best_momentum)
print("Weight Decay  :", best_wd)


In [ ]:
final_model = YOLO("yolov10n.pt")

final_results = final_model.train(
    data=DATA_YAML,
    epochs=150,
    imgsz=640,
    batch=16,
    optimizer="SGD",
    lr0=float(best_lr),
    momentum=float(best_momentum),
    weight_decay=float(best_wd),
    cos_lr=True,
    close_mosaic=10,
    patience=30,
    project="pcb_yolov10",
    name="pso_optimized"
)


In [ ]:
final_metrics = final_model.val(data=DATA_YAML)
print(final_metrics)


In [ ]:
final_model.predict(
    source=os.path.join(DATASET_PATH, "test/images"),
    conf=0.4,
    save=True,
    project="pcb_yolov10",
    name="inference_results"
)


In [ ]:
from IPython.display import Image, display
import glob

pred_images = glob.glob("pcb_yolov10/inference_results/*.jpg")
display(Image(pred_images[0]))
